In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("nifty_50_fundamentals_5years.csv")

def clean_numeric(col):
    return pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('%', ''), errors='coerce')

columns_to_clean = ["Sales+", "Operating Profit", "OPM %", "Interest", "Net Profit+", "EPS in Rs"]
for col in columns_to_clean:
    df[col] = clean_numeric(col)

df["Net Profit Margin %"] = (df["Net Profit+"] / df["Sales+"]) * 100
df["Interest Coverage"] = df["Operating Profit"] / df["Interest"]

df_clean = df.dropna(subset=["Sales+", "Operating Profit", "Net Profit+", "EPS in Rs", "OPM %"])

agg_df = df_clean.groupby("Ticker").agg({
    "Sales+": ["mean", "std"],
    "Operating Profit": "mean",
    "OPM %": "mean",
    "Net Profit+": "mean",
    "EPS in Rs": ["mean", "std"],
    "Net Profit Margin %": "mean",
    "Interest Coverage": "mean"
})
agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
agg_df.reset_index(inplace=True)


def classify_eps_stability_missing(std):
    if pd.isna(std):
        return "Data Unavailable"
    elif std < 2:
        return "Excellent"
    elif std < 5:
        return "Good"
    elif std < 10:
        return "Bad"
    else:
        return "Risky"

def classify_net_margin_missing(margin):
    if pd.isna(margin):
        return "Data Unavailable"
    elif margin > 15:
        return "Excellent"
    elif margin > 10:
        return "Good"
    elif margin > 5:
        return "Bad"
    else:
        return "Risky"

def classify_opm_missing(opm):
    if pd.isna(opm):
        return "Data Unavailable"
    elif opm > 20:
        return "Excellent"
    elif opm > 10:
        return "Good"
    elif opm > 5:
        return "Bad"
    else:
        return "Risky"

def classify_interest_coverage_missing(coverage):
    if pd.isna(coverage):
        return "Data Unavailable"
    elif coverage > 10:
        return "Excellent"
    elif coverage > 3:
        return "Good"
    elif coverage > 1:
        return "Bad"
    else:
        return "Risky"

def classify_sales_volatility_missing(std, mean):
    if pd.isna(std) or pd.isna(mean) or mean == 0:
        return "Data Unavailable"
    ratio = std / mean
    if ratio < 0.1:
        return "Excellent"
    elif ratio < 0.2:
        return "Good"
    elif ratio < 0.3:
        return "Bad"
    else:
        return "Risky"

agg_df["EPS_Stability"] = agg_df["EPS in Rs_std"].apply(classify_eps_stability_missing)
agg_df["Net_Margin_Status"] = agg_df["Net Profit Margin %_mean"].apply(classify_net_margin_missing)
agg_df["OPM_Status"] = agg_df["OPM %_mean"].apply(classify_opm_missing)
agg_df["Interest_Coverage_Status"] = agg_df["Interest Coverage_mean"].apply(classify_interest_coverage_missing)
agg_df["Sales_Stability"] = agg_df.apply(lambda row: classify_sales_volatility_missing(row["Sales+_std"], row["Sales+_mean"]), axis=1)

def score_rating_missing(row):
    scores = {
        "Excellent": 3,
        "Good": 2,
        "Bad": 1,
        "Risky": 0,
        "Data Unavailable": -1
    }
    total = sum([
        scores.get(row["EPS_Stability"], scores["Data Unavailable"]),
        scores.get(row["Net_Margin_Status"], scores["Data Unavailable"]),
        scores.get(row["OPM_Status"], scores["Data Unavailable"]),
        scores.get(row["Interest_Coverage_Status"], scores["Data Unavailable"]),
        scores.get(row["Sales_Stability"], scores["Data Unavailable"])
    ])


    if "Data Unavailable" in [row["EPS_Stability"], row["Net_Margin_Status"], row["OPM_Status"],
                              row["Interest_Coverage_Status"], row["Sales_Stability"]]:
        return "Data Unavailable"
    elif total >= 13:
        return "Excellent"
    elif total >= 9:
        return "Good"
    elif total >= 5:
        return "Bad"
    else:
        return "Risky"

agg_df["Overall_Rating"] = agg_df.apply(score_rating_missing, axis=1)

agg_df.to_csv("nifty50_classification_output.csv", index=False)

print("\nAggregated Data and Classifications for all Nifty 50 Companies:")
print(agg_df[["Ticker", "EPS_Stability", "Net_Margin_Status", "OPM_Status",
              "Interest_Coverage_Status", "Sales_Stability", "Overall_Rating"]].to_string())

FileNotFoundError: [Errno 2] No such file or directory: 'nifty_50_fundamentals_5years.csv'